In [1]:
import pandas as pd
import numpy as np

In [2]:
pd.set_option('display.max_rows', None)

In [3]:
given_test_df = pd.read_csv('data/test.csv')
test_df = pd.DataFrame(given_test_df['postcode'].unique())

In [4]:
given_test_df.head()

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,...,Light_Conditions,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country
0,14,13,2,0,06/10/13,6,13:28,218,E10000032,4,...,Daylight: Street light present,Snowing without high winds,Dry,None,None,2,Yes,England,HX2 8WH,United Kingdom
1,17,13,2,0,22/04/13,7,9:30,157,E10000034,6,...,Daylight: Street light present,Fine without high winds,Dry,None,None,1,No,England,RM8 1DD,United Kingdom
2,21,13,2,0,27/09/13,3,19:10,155,E09000012,3,...,Daylight: Street light present,Fine without high winds,Wet/Damp,None,None,1,Yes,England,SE23 1NH,United Kingdom
3,23,13,2,0,13/03/13,4,9:19,26,E10000016,4,...,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,England,HU10 7QS,United Kingdom
4,28,14,2,0,13/06/13,1,14:59,6,E08000012,4,...,Daylight: Street light present,Fine without high winds,Dry,None,None,1,Yes,England,BD23 5JL,United Kingdom


In [ ]:
test_df = test_df.rename({0:'postcode'}, axis = 1)
print(test_df.shape)
test_df.head()

In [ ]:

def postcode_short_apply(x):
    if x.find(' ') != -1:
        return x[:x.find(' ') +2]
    else:
        return x

def postcode_short_short_apply(x):
    if x.find(' ') != -1:
        return x[:x.find(' ')]
    else:
        return x

test_df['postcode_short'] = test_df['postcode'].apply(postcode_short_apply)
test_df['postcode_short_1'] = test_df['postcode'].apply(postcode_short_short_apply)

#### General Functions

In [5]:
from sklearn.preprocessing import MultiLabelBinarizer

In [6]:
#Multi Label Binarizer function

def do_multilabelbinarizer(series, col):
    mlb = MultiLabelBinarizer()
    series_index = series.index
    t = pd.DataFrame(mlb.fit_transform(series))
    t.index = series_index
    print(mlb.classes_)
    t.columns = [col + '_{0}'.format(col_class) for col_class in mlb.classes_]
    return t
    

In [ ]:
def do_statistical(series,temp_df, col):
    series_index = series.index
    t = pd.DataFrame()
    t.index = series_index

    t[col + '_min'] = temp_df.groupby('postcode')[col].min()
    t[col + '_mean'] = temp_df.groupby('postcode')[col].mean()
    t[col + '_max'] = temp_df.groupby('postcode')[col].max()
    return t


In [ ]:
def do_statistical_sum(series,temp_df, col):
    series_index = series.index
    t = pd.DataFrame()
    t.index = series_index
    # print(t)
    # print(temp_df.groupby('postcode')[col].min())
    t[col + '_sum'] = temp_df.groupby('postcode')[col].sum()
    return t

In [ ]:
def do_or_and(series,temp_df, col):
    series_index = series.index
    t = pd.DataFrame()
    t.index = series_index
    # print(t)
    # print(temp_df.groupby('postcode')[col].any())
    t[col + '_or'] = temp_df.groupby('postcode')[col].any().replace(to_replace=[False, True], value=[0, 1])
    t[col + '_and'] = temp_df.groupby('postcode')[col].all().replace(to_replace=[False, True], value=[0, 1])
    return t

In [ ]:
def do_cnt_by_cat_list(series, temp_df, col):
    series_index = series.index
    t = pd.DataFrame()
    t.index = series_index
    
    t1 = temp_df.groupby(['postcode',col])['Accident_ID'].count().unstack(level=1)
    t = t1.rename({class_col: '{0}_{1}_count'.format(col,class_col) for class_col in t1.columns}, axis=1)
    return t

In [7]:
import datetime
def time_categorize(x):
    try:
        if np.isnan(float(x)):
            return np.nan
    except:
        pass
    x = datetime.datetime.strptime(x,'%H:%M').time()
    if datetime.time(0,0,0) <= x <= datetime.time(8,0,0):
        return '0_8'
    if datetime.time(8,0,1) <= x <= datetime.time(16,0,0):
        return '8_16'
    if datetime.time(16,0,1) <= x <= datetime.time(23,59,59):
        return '16_24'
    


### Population IDVs

In [ ]:
population_df = pd.read_feather('data_feather/population_df_cleaned.feather')

In [ ]:
test_df = test_df.merge(population_df.groupby('postcode_cleaned').mean(), how = 'left', left_on='postcode_short', right_on='postcode_cleaned')
test_df.shape

### RoadNetwork IDVs

In [ ]:
roads_network_df_cleaned = pd.read_feather('data_feather/roads_network_df_cleaned.feather')
roads_network_df_cleaned = roads_network_df_cleaned.set_index('index')

In [ ]:
t1 = test_df.merge(roads_network_df_cleaned, how = 'left', left_on='postcode_short_1', right_index=True)
t2 = t1.set_index('postcode_short')
t2.update(roads_network_df_cleaned)
t2 = t2.reset_index()
t2 = t2.set_index('postcode')
t2.update(roads_network_df_cleaned)
t2 = t2.reset_index()
t2.shape

In [ ]:
test_df = t2.copy()
test_df.shape

In [ ]:
test_df.head()

### Test DF

In [ ]:
gt_mlb_col_list = ['Day_of_Week','1st_Road_Class', '2nd_Road_Class', 'Pedestrian_Crossing-Human_Control','Pedestrian_Crossing-Physical_Facilities',	'Light_Conditions', 'Weather_Conditions', 'Road_Surface_Conditions', 'Special_Conditions_at_Site', 'Carriageway_Hazards', 'state', 'Road_Type', 'Speed_limit', 'Urban_or_Rural_Area' ]
gt_statistical_calc_list = ['Police_Force', 'Number_of_Vehicles', ]
gt_or_and_list = ['Did_Police_Officer_Attend_Scene_of_Accident']

gt_cnt_by_cat_list = ['Road_Type', 'Speed_limit', 'Urban_or_Rural_Area']
gt_special_columns = ['Police_Force_sum/Number_of_Vehicles_sum', 'Police_Force_mean/Number_of_Vehicles_mean','Time_category', '1st_Road_Number',]


In [8]:
given_test_df_cleaned = pd.DataFrame(index = given_test_df['postcode'].unique())

In [9]:
given_test_df_cleaned.shape

(49772, 0)

### Combine and create test
##### For the given_test_cleaned, combine it with train_data since some are common and then create the idvs for them

In [10]:
#Joining with train set
given_train_df = pd.read_csv('data/train.csv')
given_train_df.shape, given_test_df.shape

((478741, 27), (121259, 27))

In [11]:
#run this only if you need to join and create test data
given_test_df = pd.concat([given_test_df, given_train_df])
given_test_df.shape

(600000, 27)

### As Usual

In [ ]:
for col in gt_mlb_col_list:
    t = do_multilabelbinarizer(given_test_df.fillna('').groupby('postcode')[col].unique(), col)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_index = True, right_index=True, how = 'left')


given_test_df_cleaned.head()

In [ ]:
for col in gt_statistical_calc_list:
    t = do_statistical(given_test_df.groupby('postcode')[col].unique(), given_test_df, col)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_index = True, right_index=True, how = 'left')

given_test_df_cleaned.head()

In [ ]:
for col in gt_statistical_calc_list:
    t = do_statistical_sum(given_test_df.groupby('postcode')[col].unique(), given_test_df, col)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_index = True, right_index=True, how = 'left')

given_test_df_cleaned.head()

In [ ]:
for col in gt_or_and_list:
    t = do_or_and(given_test_df.groupby('postcode')[col].unique(), given_test_df, col)
    # print(t)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_index = True, right_index=True, how = 'left')

given_test_df_cleaned.head()

In [ ]:
for col in gt_cnt_by_cat_list:
    t = do_cnt_by_cat_list(given_test_df.groupby('postcode')[col].unique(), given_test_df, col)
    # print(t)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_index = True, right_index=True, how = 'left')

given_test_df_cleaned.head()

In [ ]:

gt_special_columns = ['Police_Force_sum/Number_of_Vehicles_sum', 'Police_Force_mean/Number_of_Vehicles_mean','Time_category', '1st_Road_Number',]
given_test_df_cleaned['Police_Force_sum/Number_of_Vehicles_sum'] = given_test_df_cleaned['Police_Force_sum']/given_test_df_cleaned['Number_of_Vehicles_sum']
given_test_df_cleaned['Police_Force_mean/Number_of_Vehicles_mean'] = given_test_df_cleaned['Police_Force_mean']/given_test_df_cleaned['Number_of_Vehicles_mean']

In [ ]:
given_test_df['Time_category']  = given_test_df['Time'].apply(time_categorize)
given_test_df.head()

In [ ]:
for col in ['Time_category']:
    t = do_cnt_by_cat_list(given_test_df.groupby('postcode')[col].unique(), given_test_df, col)
    # print(t)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_index = True, right_index=True, how = 'left')

given_test_df_cleaned.head()

In [ ]:
road_num_count = given_test_df['1st_Road_Number'].value_counts().rename('1st_Road_Number_count') #just count 
t = given_test_df.groupby(['postcode','1st_Road_Number'])['Accident_ID'].count().reset_index().merge(road_num_count, how='left', left_on='1st_Road_Number', right_index=True) #merging with postcode
t['1st_Road_Number_count>100'] = (t['1st_Road_Number_count']>100).replace(to_replace=[False,True], value=(0,1))#calculating how many are count > 100 for each postcode
given_test_df_cleaned = given_test_df_cleaned.merge(t.groupby('postcode').sum()['1st_Road_Number_count>100'], how='left', left_index=True, right_index=True)

In [ ]:
given_test_df_cleaned.shape

In [ ]:
given_test_df_cleaned.reset_index().rename({'index':'postcode'}).to_feather('data_feather/given_test_df_cleaned_combined_1.feather')

In [ ]:
test_df = test_df.merge(given_test_df_cleaned,how='left', right_index=True, left_on='postcode')
print(test_df.shape)
test_df.head()

In [ ]:
train_df = pd.read_feather('data_feather/train_df_3.feather')
train_df.head()

### Imputation

In [ ]:
print(test_df.isna().sum()[test_df.isna().sum() > 0])


In [ ]:
import joblib

In [ ]:
#Median
median_categories = ['All usual residents', 'Males', 'Females', 'Lives in a household', 'Lives in a communal establishment', 'Schoolchild or full-time student aged 4 and over at their non term-time address', 'Road_Type_Single carriageway_count', 'Speed_limit_30_count', 'Urban_or_Rural_Area_1_count', 'Urban_or_Rural_Area_2_count', 'Time_category_16_24_count', 'Time_category_8_16_count', ]
imp_median = joblib.load('data_feather/imp_median.pkl')
test_df[median_categories] = imp_median.transform(test_df[median_categories])



In [ ]:
#Mean
mean_categories = ['Area (Hectares)', 'Density (number of persons per hectare)', 'length_min', 'length_mean', 'length_max', 'distance to the nearest point on rd_min', 'distance to the nearest point on rd_mean', 'distance to the nearest point on rd_max', ]
imp_mean = joblib.load('data_feather/imp_mean.pkl')
test_df[mean_categories] = imp_mean.transform(test_df[mean_categories])


In [ ]:
#Impute NaN with 0
zero_categories = ['roadClassi_', 'roadClassi_A Road', 'roadClassi_Motorway', 'roadFuncti_', 'roadFuncti_A Road', 'roadFuncti_Motorway', 'formOfWay_', 'formOfWay_Collapsed Dual Carriageway', 'formOfWay_Dual Carriageway', 'formOfWay_Roundabout', 'formOfWay_Single Carriageway', 'formOfWay_Slip Road', 'primaryRou_or', 'primaryRou_and', 'Road_Type_Dual carriageway_count', 'Road_Type_One way street_count', 'Road_Type_Roundabout_count', 'Road_Type_Single carriageway_count', 'Road_Type_Slip road_count', 'Road_Type_Unknown_count', 'Speed_limit_20_count', 'Speed_limit_30_count', 'Speed_limit_40_count', 'Speed_limit_50_count', 'Speed_limit_60_count', 'Speed_limit_70_count', 'Urban_or_Rural_Area_1_count', 'Urban_or_Rural_Area_2_count','Time_category_0_8_count', 'Time_category_16_24_count', 'Time_category_8_16_count']
imp_zero = joblib.load('data_feather/imp_zero.pkl')
test_df[zero_categories] = imp_zero.transform(test_df[zero_categories])


In [ ]:
print(test_df.isna().sum()[test_df.isna().sum() > 0])


In [ ]:
test_df.to_feather('data/test_df_processed_combined_1.feather')

In [ ]:
test_df.head()

In [ ]:
given_test_df.head()

In [ ]:
for col in train_df.columns:
    print(col)

### Added New columns

In [ ]:
train_df = pd.read_feather('data_feather/train_df_4.feather')
print(train_df.shape)
train_df.head()

In [12]:
# test_df = pd.read_feather('data_feather/test_df_processed.feather')
test_df = pd.read_feather('data_feather/test_df_processed_combined_1.feather')
print(test_df.shape)
test_df.head()

(49772, 139)


,postcode,postcode_short,postcode_short_1,All usual residents,Males,Females,Lives in a household,Lives in a communal establishment,Schoolchild or full-time student aged 4 and over at their non term-time address,Area (Hectares),...,Speed_limit_60_count,Speed_limit_70_count,Urban_or_Rural_Area_1_count,Urban_or_Rural_Area_2_count,Police_Force_sum/Number_of_Vehicles_sum,Police_Force_mean/Number_of_Vehicles_mean,Time_category_0_8_count,Time_category_16_24_count,Time_category_8_16_count,1st_Road_Number_count>100
0,HX2 8WH,HX2 8,HX2,7032.0,3411.0,3621.0,7027.0,5.0,37.0,577.28,...,1.000000,1.155255,9.0,3.0,5.545455,5.545455,1.000000,5.0,6.0,9
1,RM8 1DD,RM8 1,RM8,11371.0,5450.0,5921.0,11314.0,57.0,110.0,221.90,...,1.560634,1.155255,5.0,2.0,10.500000,10.500000,1.000000,2.0,4.0,4
2,SE23 1NH,SE23 1,SE23,8639.0,4196.0,4443.0,8611.0,28.0,80.0,97.97,...,1.560634,1.155255,4.0,2.0,5.000000,5.000000,1.298796,2.0,2.0,2
3,HU10 7QS,HU10 7,HU10,8147.0,3981.0,4166.0,8030.0,117.0,132.0,812.91,...,1.560634,1.155255,4.0,2.0,11.571429,11.571429,1.298796,1.0,3.0,2
4,BD23 5JL,BD23 5,BD23,3485.0,1654.0,1831.0,3376.0,109.0,76.0,26528.38,...,1.560634,1.155255,12.0,1.0,8.880000,8.880000,1.298796,3.0,10.0,11


In [13]:
given_test_df['Time_category']  = given_test_df['Time'].apply(time_categorize)
given_test_df.head()

,Accident_ID,Police_Force,Number_of_Vehicles,Number_of_Casualties,Date,Day_of_Week,Time,Local_Authority_(District),Local_Authority_(Highway),1st_Road_Class,...,Weather_Conditions,Road_Surface_Conditions,Special_Conditions_at_Site,Carriageway_Hazards,Urban_or_Rural_Area,Did_Police_Officer_Attend_Scene_of_Accident,state,postcode,country,Time_category
0,14,13,2,0,06/10/13,6,13:28,218,E10000032,4,...,Snowing without high winds,Dry,None,None,2,Yes,England,HX2 8WH,United Kingdom,8_16
1,17,13,2,0,22/04/13,7,9:30,157,E10000034,6,...,Fine without high winds,Dry,None,None,1,No,England,RM8 1DD,United Kingdom,8_16
2,21,13,2,0,27/09/13,3,19:10,155,E09000012,3,...,Fine without high winds,Wet/Damp,None,None,1,Yes,England,SE23 1NH,United Kingdom,16_24
3,23,13,2,0,13/03/13,4,9:19,26,E10000016,4,...,Fine without high winds,Dry,None,None,1,Yes,England,HU10 7QS,United Kingdom,8_16
4,28,14,2,0,13/06/13,1,14:59,6,E08000012,4,...,Fine without high winds,Dry,None,None,1,Yes,England,BD23 5JL,United Kingdom,8_16


In [14]:
#Only for combined
given_test_df = given_test_df.reset_index().drop('index', axis = 1)

In [15]:
from category_encoders import TargetEncoder
import joblib
t_encoder = joblib.load( 't_encoder.pkl')

t_enc_list = ['1st_Road_Class','Urban_or_Rural_Area','Speed_limit','Day_of_Week','Time_category',]
t_enc_out = t_encoder.transform(given_test_df[t_enc_list])
t_enc_out.shape

(600000, 5)

In [16]:
t_enc_out = t_enc_out.rename({col: col + '_t_enc' for col in t_enc_out.columns}, axis = 1)
t_enc_out[:10]

,1st_Road_Class_t_enc,Urban_or_Rural_Area_t_enc,Speed_limit_t_enc,Day_of_Week_t_enc,Time_category_t_enc
0,1.467137,1.473621,1.472095,1.463941,1.464339
1,1.449474,1.456742,1.456428,1.461146,1.464339
2,1.477160,1.456742,1.488362,1.463686,1.462345
3,1.467137,1.456742,1.456428,1.464252,1.464339
4,1.467137,1.456742,1.456428,1.458004,1.464339
5,1.449474,1.456742,1.456428,1.461146,1.464339
6,1.477160,1.473621,1.488437,1.463941,1.464339
7,1.477160,1.473621,1.456428,1.465034,1.463926
8,1.467137,1.473621,1.472095,1.464252,1.462345
9,1.477160,1.473621,1.472095,1.463941,1.462345


In [17]:
given_test_df = given_test_df.merge( t_enc_out, left_index=True, right_index=True)
given_test_df.shape

(600000, 33)

In [25]:

given_test_df_cleaned = given_test_df_cleaned.reset_index().rename({'index':'postcode'}, axis = 1)

In [26]:
given_test_df_cleaned.head()

,postcode
0,HX2 8WH
1,RM8 1DD
2,SE23 1NH
3,HU10 7QS
4,BD23 5JL


In [27]:
given_test_df_cleaned = given_test_df_cleaned.merge(given_test_df.groupby('postcode').mean()[[col+ '_t_enc' for col in t_enc_list]], how='left', left_on = 'postcode', right_index=True)
given_test_df_cleaned.shape

(49772, 6)

In [28]:
#no. of accidents
given_test_df_cleaned = given_test_df_cleaned.merge(given_test_df.groupby('postcode').count()['Accident_ID'].rename('No. Of Accidents'), how='left', left_on = 'postcode', right_index=True)
given_test_df_cleaned.shape

(49772, 7)

In [29]:
for col in ['Time_category']:
    t = do_multilabelbinarizer(given_test_df.fillna('').groupby('postcode')[col].unique(), col)
    # print(t)
    given_test_df_cleaned = given_test_df_cleaned.merge(t, left_on='postcode', right_index=True, how = 'left')

given_test_df_cleaned.head()

#We don't need Time_category_
given_test_df_cleaned = given_test_df_cleaned.drop([ 'Time_category_'	],axis = 1)
given_test_df_cleaned.shape

['' '0_8' '16_24' '8_16']


(49772, 10)

In [30]:
given_test_df_cleaned = given_test_df_cleaned.merge(test_df[['Day_of_Week_1','Day_of_Week_2','Day_of_Week_3','Day_of_Week_4','Day_of_Week_5','Day_of_Week_6','Day_of_Week_7','postcode']], 
left_on='postcode', right_on='postcode')
given_test_df_cleaned.shape

(49772, 17)

In [31]:
for date_col in ['Day_of_Week_1','Day_of_Week_2','Day_of_Week_3','Day_of_Week_4','Day_of_Week_5','Day_of_Week_6','Day_of_Week_7',]:
    for time_col in ['Time_category_0_8','Time_category_16_24','Time_category_8_16']:
        given_test_df_cleaned[date_col + '_and_' + time_col] = given_test_df_cleaned[date_col] & given_test_df_cleaned[time_col]

given_test_df_cleaned.shape

(49772, 38)

In [32]:
test_df = test_df.merge(given_test_df_cleaned[[col+ '_t_enc' for col in t_enc_list] + ['No. Of Accidents',] + ['Time_category_0_8', 'Time_category_16_24','Time_category_8_16', 'Day_of_Week_1_and_Time_category_0_8', 'Day_of_Week_1_and_Time_category_16_24', 'Day_of_Week_1_and_Time_category_8_16', 'Day_of_Week_2_and_Time_category_0_8', 'Day_of_Week_2_and_Time_category_16_24', 'Day_of_Week_2_and_Time_category_8_16', 'Day_of_Week_3_and_Time_category_0_8', 'Day_of_Week_3_and_Time_category_16_24', 'Day_of_Week_3_and_Time_category_8_16', 'Day_of_Week_4_and_Time_category_0_8', 'Day_of_Week_4_and_Time_category_16_24', 'Day_of_Week_4_and_Time_category_8_16', 'Day_of_Week_5_and_Time_category_0_8', 'Day_of_Week_5_and_Time_category_16_24', 'Day_of_Week_5_and_Time_category_8_16', 'Day_of_Week_6_and_Time_category_0_8', 'Day_of_Week_6_and_Time_category_16_24', 'Day_of_Week_6_and_Time_category_8_16', 'Day_of_Week_7_and_Time_category_0_8', 'Day_of_Week_7_and_Time_category_16_24', 
'Day_of_Week_7_and_Time_category_8_16','postcode']], how='left', left_on = 'postcode', right_on = 'postcode')
print(test_df.shape)
test_df.head()

(49772, 169)


,postcode,postcode_short,postcode_short_1,All usual residents,Males,Females,Lives in a household,Lives in a communal establishment,Schoolchild or full-time student aged 4 and over at their non term-time address,Area (Hectares),...,Day_of_Week_4_and_Time_category_8_16,Day_of_Week_5_and_Time_category_0_8,Day_of_Week_5_and_Time_category_16_24,Day_of_Week_5_and_Time_category_8_16,Day_of_Week_6_and_Time_category_0_8,Day_of_Week_6_and_Time_category_16_24,Day_of_Week_6_and_Time_category_8_16,Day_of_Week_7_and_Time_category_0_8,Day_of_Week_7_and_Time_category_16_24,Day_of_Week_7_and_Time_category_8_16
0,HX2 8WH,HX2 8,HX2,7032.0,3411.0,3621.0,7027.0,5.0,37.0,577.28,...,1,1,1,1,1,1,1,1,1,1
1,RM8 1DD,RM8 1,RM8,11371.0,5450.0,5921.0,11314.0,57.0,110.0,221.90,...,0,1,0,1,1,0,1,1,0,1
2,SE23 1NH,SE23 1,SE23,8639.0,4196.0,4443.0,8611.0,28.0,80.0,97.97,...,0,0,1,1,0,0,0,0,0,0
3,HU10 7QS,HU10 7,HU10,8147.0,3981.0,4166.0,8030.0,117.0,132.0,812.91,...,1,0,0,0,0,1,1,0,1,1
4,BD23 5JL,BD23 5,BD23,3485.0,1654.0,1831.0,3376.0,109.0,76.0,26528.38,...,0,0,1,1,0,1,1,0,1,1


In [33]:
test_df.to_feather('data_feather/test_df_processed_combined_2.feather')
# test_df.to_feather('data_feather/test_df_processed_2.feather')

In [ ]:
test_df = pd.read_feather('data_feather/test_df_processed_2.feather')
test_df.shape

In [ ]:
# test_df.to_feather('data_feather/test_df_processed_2.feather')